In [23]:
import yfinance as yf
import pandas as pd
import itertools
import os

In [24]:
# currency in which the simulation is run
analysis_currency = "EUR"

# start and end date of the simulation
# if start_date is before the first available date of the securities, the first available date will be used
start_date = "2019-08-01"
end_date = "2023-11-01"
# end_date = datetime.datetime.today().strftime("%Y-%m-%d")

# tickers and the corresponding currencies of securities to download from yahoo finance
tickers_and_currencies = {
    "VWCE.DE": "EUR",
    "ISAC.L": "USD",
    "VAGP.L": "GBP",
    "AGGH.MI": "EUR",
    "4GLD.DE": "EUR",
    "IGLN.L": "USD",
}

# amount of expense (transaction fee) for each transaction
expense = 1

# approximate amount of money to invest in each transaction
amount = 1000

# precision is a step in percentage points for testing different weights (precision = 1 means 0%, 1%, 2%, etc., precision = 3 means 0%, 3%, 6%, etc.)
precision = 10

# which column to use for open, high, low, close prices
ohlc = "close"

# every n months a transaction will be made
n_months = 1

In [25]:
# take securities names from tickers_and_currencies dictionary
securities = [security_name.split(".")[0] for security_name in tickers_and_currencies]

# take tickers and currencies from tickers_and_currencies dictionary
tickers = [*tickers_and_currencies.keys()]
currencies_securities = [*tickers_and_currencies.values()]

# take all currencies
currencies = currencies_securities + [analysis_currency]

# take distinct currencies
distinct_currencies = list(set(currencies))

# convert ohlc variable to valid format
ohlc = ohlc[0].upper() + ohlc[1:].lower()

In [26]:
# download securities data from yahoo finance
yahoo_securities_data = yf.download(tickers, period="max")[ohlc]

# make yahoo_securities_data dataframe
securities_data = pd.DataFrame(yahoo_securities_data)

# set columns order to a specified one in order to correctly set columns names later
securities_data = securities_data[tickers]

# set index name to 'Date' and set columns names to securities names
securities_data.index.name = "Date"
securities_data.columns = securities

[*********************100%***********************]  6 of 6 completed


In [27]:
# create list of currency pairs to download exchange rates for
distinct_currency_pairs = [
    currency + analysis_currency for currency in distinct_currencies
]

# remove currency pair which is the same as analysis currency
distinct_currency_pairs.remove(analysis_currency * 2)

# if currency pairs are specified then download exchange rates for them else set exchange rates to None
distinct_currency_pairs_format = [
    currency + "=X" for currency in distinct_currency_pairs
]

# download currency data from yahoo finance
yahoo_currency_data = yf.download(distinct_currency_pairs_format, period="max")[ohlc]

# make yahoo_currency_data dataframe
exchange_rates = pd.DataFrame(yahoo_currency_data)

# set columns order to a specified one in order to correctly set columns names later
exchange_rates = exchange_rates[distinct_currency_pairs_format]
exchange_rates.columns = distinct_currency_pairs

# if there is analysis currency in distinct currencies then add column with exchange rates equal to 1.0
if distinct_currencies.index(analysis_currency) != -1:
    analysis_currency_exchange_rates = pd.Series(
        [1.0 for _ in range(len(securities_data.index))],
        index=securities_data.index,
        name=analysis_currency * 2,
    )
    exchange_rates = pd.concat(
        [exchange_rates, analysis_currency_exchange_rates], axis=1
    )

[*********************100%***********************]  2 of 2 completed


In [28]:
# make a copy of securities_data dataframe to use it for simulation
# copy to reuse securities_data dataframe if needed without downloading data again
simulation_data = securities_data.copy()

# convert simulation_data dataframe to the currency of analysis
for ticker, currency in tickers_and_currencies.items():
    security_name = ticker.split(".")[0]
    currency_pair = currency + analysis_currency
    simulation_data[security_name] = (
        simulation_data[security_name] * exchange_rates[currency_pair]
    )

In [29]:
# drop rows with NaN values as it is not possible to simulate portfolio with them
simulation_data = simulation_data.dropna()

In [30]:
securities_names = [name[:4] for name in tickers]

securities_unit_value = [name + "_UNIT_VALUE" for name in securities_names]
securities_count = [name + "_COUNT" for name in securities_names]
securities_value = [name + "_VALUE" for name in securities_names]
securities_expense = [name + "_EXPENSE" for name in securities_names]
securities_buy_moment_expense = [
    name + "_BUY_MOMENT_EXPENSE" for name in securities_names
]
securities_buy_moment_count = [name + "_BUY_MOMENT_COUNT" for name in securities_names]
securities_profit = [name + "_PROFIT" for name in securities_names]

# portfolio columns names
portfolio_count = "PORTFOLIO_COUNT"
portfolio_value = "PORTFOLIO_VALUE"
portfolio_expense = "PORTFOLIO_EXPENSE"
portfolio_profit = "PORTFOLIO_PROFIT"
portfolio_profit_percentage = "PORTFOLIO_PROFIT_PERCENTAGE"
portfolio_drawdown = "PORTFOLIO_DRAWDOWN"
portfolio_cagr = "PORTFOLIO_CAGR"

In [31]:
# change columns names to the ones with suffix
simulation_data.columns = securities_unit_value

# add columns with suffix and set values to 0
simulation_data[securities_count] = 0

In [32]:
# function that generates buy transaction dates every n months
def generate_buy_transaction_dates(simulation_data):
    buy_transaction_dates = []
    dates_groups = simulation_data.groupby(
        [simulation_data.index.year, simulation_data.index.month]
    )

    for i, (_, group) in enumerate(dates_groups):
        if i % n_months == 0:
            buy_transaction_dates.append(group.sample(1).index[0])

    return buy_transaction_dates

In [33]:
# generate buy transaction dates
buy_transaction_dates = generate_buy_transaction_dates(simulation_data)

In [34]:
# take only rows with dates greater than the first buy transaction date
simulation_data = simulation_data.loc[
    simulation_data.index >= buy_transaction_dates[0]
].copy()

In [35]:
# dictionary with statistics of profit for each iteration
results_profit_describe = {}

# dictionary with statistics of profit percentage for each iteration
results_profit_percentage_describe = {}

# dictionary with statistics of drawdown for each iteration
results_drawdown_describe = {}

In [ ]:
sequence = [i / 100 for i in range(0, 101, precision)]
args = [sequence] * len(tickers)

In [ ]:
for weights in itertools.product(*args):
    # if sum of weights is not equal to 1 then continue
    if sum(weights) != 1:
        continue

    # make a copy of simulation_data dataframe to use it for simulation
    simulation_data_copy = simulation_data.copy()

    # loop through buy transaction dates
    for date in buy_transaction_dates:
        # cumulative sum of securities counts to get current counts
        counts_cumsum = simulation_data_copy[securities_count].cumsum()

        # cumulative sum of securities counts to get counts for a given date
        counts_cumsum = simulation_data_copy[securities_count].cumsum()

        # get securities counts for a given date
        counts_for_date = counts_cumsum.loc[date]

        # get securities unit values for a given date
        unit_values_for_date = simulation_data_copy.loc[date, securities_unit_value]

        # get current values of securities by multiplying currect counts and unit values
        current_values = counts_for_date.values * unit_values_for_date.values

        # get current values sum
        current_values_sum = current_values.sum()

        # get weights deviations by subtracting current share of each security from the target share
        # if current_values_sum is not 0 else set weights deviations to list of 0s
        if current_values_sum:
            weights_deviations = [
                weight - current_value / current_values_sum
                for weight, current_value in zip(weights, current_values)
            ]
        else:
            weights_deviations = [0] * len(weights)

        # get weights model plus deviations by adding weights and their deviations
        # positive weights deviations mean that the security is underweighted and vice versa
        weights_model_plus_deviations = [
            weight + deviation for weight, deviation in zip(weights, weights_deviations)
        ]

        # get security amounts by multiplying weights model plus deviations by the amount
        security_amounts = [amount * weight for weight in weights_model_plus_deviations]

        # get buy counts by dividing security amounts by securities unit values
        buy_counts = (
            security_amounts / simulation_data_copy.loc[date, securities_unit_value]
        )

        # round down buy_counts to the integer and add 0.1 to round up the values with 0.9 decimal part
        buy_counts = buy_counts.apply(lambda x: int(x + 0.1))

        # assign new transaction buy counts and expense to a given date in simulation_data_copy dataframe
        simulation_data_copy.loc[date, securities_count] = buy_counts.values
        simulation_data_copy.loc[date, securities_expense] = [expense] * len(
            securities_names
        )

    # assign securities counts to expenses and buy moment counts
    simulation_data_copy[securities_expense] = simulation_data_copy[securities_count]
    simulation_data_copy[securities_buy_moment_count] = simulation_data_copy[
        securities_count
    ]

    # assign securities count cummulatives sums to counts and values
    simulation_data_copy[securities_count] = simulation_data_copy[
        securities_count
    ].cumsum()
    simulation_data_copy[securities_value] = simulation_data_copy[securities_count]

    # multiply securities values by securities unit values to get securities values based on transaction counts
    for security_value, security_unit_value in zip(
        securities_value, securities_unit_value
    ):
        simulation_data_copy[security_value] = (
            simulation_data_copy[security_value]
            * simulation_data_copy[security_unit_value]
        )

    # multiply securities expenses by securities unit values to get securities expenses based on transaction counts and add expense variable
    for security_expense, security_unit_value in zip(
        securities_expense, securities_unit_value
    ):
        simulation_data_copy[security_expense] = (
            simulation_data_copy[security_expense]
            * simulation_data_copy[security_unit_value]
        )
        simulation_data_copy.loc[
            simulation_data_copy[security_expense] > 0, security_expense
        ] += expense

    # assign securities expenses to buy moment expenses and calculate cumulative sum of securities expenses
    simulation_data_copy[securities_buy_moment_expense] = simulation_data_copy[
        securities_expense
    ]
    simulation_data_copy[securities_expense] = simulation_data_copy[
        securities_expense
    ].cumsum()

    # calculate securities profits
    for security_profit, security_value, security_expense in zip(
        securities_profit, securities_value, securities_expense
    ):
        simulation_data_copy[security_profit] = (
            simulation_data_copy[security_value]
            - simulation_data_copy[security_expense]
        )

    # calculate portfolio values, expenses, profits and profit percentages
    simulation_data_copy[portfolio_value] = simulation_data_copy[securities_value].sum(
        axis=1
    )
    simulation_data_copy[portfolio_expense] = simulation_data_copy[
        securities_expense
    ].sum(axis=1)
    simulation_data_copy[portfolio_profit] = (
        simulation_data_copy[portfolio_value] - simulation_data_copy[portfolio_expense]
    )
    simulation_data_copy[portfolio_profit_percentage] = (
        simulation_data_copy[portfolio_profit] / simulation_data_copy[portfolio_expense]
    ) * 100

    # calculate portfolio drawdowns
    for date in simulation_data_copy.index:
        # calculate max portfolio value for the current date
        max_value = simulation_data_copy.loc[:date, portfolio_value].max()

        # take current portfolio value
        current_value = simulation_data_copy.loc[date, portfolio_value]

        # if max_value is 0 then set drawdown to 0 else calculate drawdown
        if max_value == 0:
            simulation_data_copy.loc[date, portfolio_drawdown] = 0
            continue

        # calculate drawdown by subtracting current value from max value and dividing by max value
        simulation_data_copy.loc[date, portfolio_drawdown] = (
            current_value - max_value
        ) / max_value

    # calculate portfolio cagr
    for date in buy_transaction_dates:
        # calculate datediff between the last date and the current date
        datediff = simulation_data_copy.index[-1] - date

        # if datediff is less than 365 days then break the loop
        if datediff.days < 365:
            break

        # calculate future value by multiplying last securities unit values by securities buy moment counts for the current date and summing them
        future_value = (
            simulation_data_copy.iloc[-1].loc[securities_unit_value].values
            * simulation_data_copy.loc[date, securities_buy_moment_count].values
        )
        future_value = sum(future_value)

        # calculate cagr by dividing future value by securities buy moment expenses sum and raising to the power of 365 / datediff.days
        simulation_data_copy.loc[date, portfolio_cagr] = (
            future_value
            / simulation_data_copy.loc[date, securities_buy_moment_expense].sum()
        ) ** (365 / datediff.days) - 1

    # -------------------- add iteration results to results dictionaries -------------------- #
    # convert weights to string to use it as a key in a results dictionary
    weights_str = str(weights)

    # add portfolio profit describe to a results dictionary
    results_profit_describe[weights_str] = simulation_data_copy[
        portfolio_profit
    ].describe()

    # add portfolio profit percentage describe to a results dictionary
    results_profit_percentage_describe[weights_str] = simulation_data_copy[
        portfolio_profit_percentage
    ].describe()

    # add portfolio max drawdown describe to a results dictionary
    results_drawdown_describe[weights_str] = simulation_data_copy[
        portfolio_drawdown
    ].describe()

    print(f"Finished iteration with weights: {weights_str}")

In [ ]:
# make dataframes from results dictionaries

df_profit_describe = pd.DataFrame.from_dict(results_profit_describe, orient="index")
df_profit_percentage_describe = pd.DataFrame.from_dict(
    results_profit_percentage_describe, orient="index"
)
df_drawdown_describe = pd.DataFrame.from_dict(results_drawdown_describe, orient="index")

In [2]:
# create results directory if it does not exist
os.makedirs("results", exist_ok=True)

# save results to csv files
df_profit_describe.to_csv(os.path.join("results", "profit_describe.csv"))
df_profit_percentage_describe.to_csv(
    os.path.join("results", "profit_percentage_describe.csv")
)
df_drawdown_describe.to_csv(os.path.join("results", "drawdown_describe.csv"))

NameError: name 'df_profit_describe' is not defined

In [37]:
# calculate mean to std ratio for profit
df_profit_mean_to_std = df_profit_describe["mean"] / df_profit_describe["std"]

In [38]:
# make a dataframe with chosen results
df_results = pd.concat(
    [df_profit_mean_to_std, df_drawdown_describe['min'], df_drawdown_describe['mean'], df_profit_percentage_describe['mean'], df_profit_percentage_describe['max'], df_profit_percentage_describe['min']], axis=1
)

# set columns names
columns_values = ['profit_mean_to_std', 'min_drawdown', 'mean_drawdown', 'mean_profit_percentage', 'max_profit_percentage', 'min_profit_percentage']
df_results.columns = columns_values

# sort values by profit mean to std ratio and min drawdown in descending order
df_results = df_results.sort_values(by=['profit_mean_to_std', 'min_drawdown'], ascending=[False, False])

In [39]:
# reset index and rename it to weights
df_results = df_results.reset_index()
df_results = df_results.rename(columns={"index": "weights"})

In [40]:
# convert weights column to list of floats
df_results['weights'] = df_results['weights'].apply(lambda x: eval(x))

# create columns with weights values for each security
df_results[securities] = pd.DataFrame(df_results['weights'].to_list(), index=df_results.index)

# drop weights column
df_results = df_results.drop(columns=['weights'])

# reorder columns
df_results = df_results[securities + columns_values]

In [41]:
# convert drawdown values to percentages
for col in securities + ['min_drawdown', 'mean_drawdown']:
    df_results[col] = df_results[col].apply(lambda x: str(round(x * 100, 2)) + '%')

In [42]:
# add % to profit percentage values
for col in ['mean_profit_percentage', 'max_profit_percentage', 'min_profit_percentage']:
    df_results[col] = df_results[col].apply(lambda x: str(round(x, 2)) + '%')

In [74]:
# add styles to dataframe to make it look better in html
styled_df = df_results.style.set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', 'lightblue'), ('color', 'black'), ('font-family', 'Arial'), ('text-align', 'center'), ('border', '10px solid lightblue')]},
        {'selector': 'tbody td', 'props': [('border', '1px solid #bdc3c7')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#ecf0f1')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': '.data', 'props': [('text-align', 'center')]},
    ])

# save styled dataframe to html file
styled_df.to_html('simulation_results.html')